Before committing and pushing to the repo, please "Clear All" jupyter outputs.

In [33]:
import pandas as pd
from pathlib import Path
from pprint import pp

In [ ]:
import ollama 
import math

In [34]:
pd.set_option('display.max_columns', 100)

In [ ]:
# prepare input/output folder paths

data_folder = Path('data')/'questionnaire-data-july-1-2024'
assert data_folder.exists(), f'{data_folder=} is missing.'

experiement_name = 'test'
main_output_folder = Path(f'output')
assert main_output_folder.exists(), f'{main_output_folder=} is missing.'
output_folder = main_output_folder/experiement_name
assert output_folder.exists(), f'{output_folder=} is missing. Please create it manually.'



# Load Data

In [ ]:

filepaths = {'course_start':data_folder/'ערבית מדוברת: מתחילים - שאלון פתיחת קורס.csv',
'mid_course': data_folder/'ערבית מדוברת: מתחילים - שאלון במהלך הקורס.csv',
'course_end': data_folder/' "ערבית מדוברת: מתחילים" - שאלון סיום קורס.csv'
}




In [ ]:
dfs = dict()

for key, file_path in filepaths.items():
    dfs[key] = pd.read_csv(file_path)

Manually define which columns are text, per df (course start / mid / end):

In [ ]:
text_columns = dict()

In [ ]:
qstr_key = 'course_start'
dfs[qstr_key].columns
text_cols = [-1,-2]
text_columns[qstr_key] = list(dfs[qstr_key].columns[text_cols])
dfs[qstr_key][text_columns[qstr_key]].sample(5)

In [ ]:
qstr_key = 'mid_course'
# dfs[qstr_key].head(3)
# dfs[qstr_key].columns
text_cols = [4,5,-1]
text_columns[qstr_key] = list(dfs[qstr_key].columns[text_cols])
dfs[qstr_key][text_columns[qstr_key]].sample(5)

In [ ]:
qstr_key = 'course_end'
# dfs[qstr_key].head(3)
text_cols = [5,6,-2]
text_columns[qstr_key] = list(dfs[qstr_key].columns[text_cols])
dfs[qstr_key][text_columns[qstr_key]].sample(5)

# Sentence Splitting

In [ ]:
import nltk 
nltk.download('punkt')

from nltk import sent_tokenize
import re

def split_into_sentences(text):
    # custom_sentence_end = r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|,|\n|\?|\!)\s'
    
    # custom_sentence_end = re.compile(custom_sentence_end)
    custom_pattern = r'[.!?\\n]'
    text = re.sub(custom_pattern, " . ", text)
    
    sentences_ = sent_tokenize(text)
    
    return sentences_

def preprocess(docs_col):
    sentences = docs_col.apply(split_into_sentences)
    sentences = sentences.explode()
    sentences.dropna(inplace=True)
    sentences = sentences.str.replace('.','')
    sentences = sentences.str.strip()
    fltr = sentences.apply(len) < 3
    sentences.drop(index=fltr[fltr].index, inplace=True)
    col_name = sentences.name
    sentences = sentences.reset_index(drop=True)
    # sentences.columns = ['record_id',col_name]
    return sentences



# Topic Modeling

In [ ]:
from bertopic import BERTopic
from bertopic.vectorizers import ClassTfidfTransformer
from sentence_transformers import SentenceTransformer
from bertopic.representation import MaximalMarginalRelevance

from hdbscan import HDBSCAN
from umap import UMAP

def set_topic_model(verbose=0):
    topic_size_ = 10
    umap_model = UMAP(n_components=16, n_neighbors=3, min_dist=0.0)
    hdbscan_model = HDBSCAN(min_cluster_size = topic_size_, gen_min_span_tree=True, prediction_data=True, min_samples=4)
    sentence_model = SentenceTransformer("imvladikon/sentence-transformers-alephbert")  # all-MiniLM-L6-v2
    ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True, bm25_weighting=True)
    representation_model = MaximalMarginalRelevance(diversity=0.1)
    # topic_model = BERTopic(embedding_model=sentence_model, representation_model=representation_model)

    bert_config = {'language':"hebrew",
                'embedding_model':sentence_model,
                'top_n_words':20,
                'n_gram_range':(1, 4),
                'min_topic_size':topic_size_,
                'nr_topics':15,
                'low_memory':False,
                'calculate_probabilities':False,
                'umap_model':umap_model,
                'hdbscan_model':hdbscan_model,
                'ctfidf_model':ctfidf_model,
                'representation_model':representation_model
                }


    topic_model = BERTopic(**bert_config)
    if verbose>8:
        pp(topic_model.get_params())
    return topic_model


# Summarization

In [ ]:
from textwrap import dedent

In [ ]:
# models = ['llama3.1:latest', 'aminadaven/dictalm2.0-instruct:f16']

def summarize(col_name: str, sentences: str, model='aminadaven/dictalm2.0-instruct:f16', verbose=0): # model in 'llama3'
    
    # system_instructions = ' '.join([
    #     f"המטרה שלך היא לתרגם לעברית ואז לסכם במדויק בעברית.",
    #     f"מצורפות תשובות התלמידים לגבי השאלה הבאה [<'{col_name.strip()}'>].",
    # f"סכם את התשובות בפסקה אחת בדיוק עם לכל היותר {max(math.ceil(math.log(len(sentences))),2)} משפטים.",
    # "הקפד לוודא שהתשובה מבוססת רק על הדעות שניתנו.",
    # "מבחינה דקדוקית, נסח את הסיכום בגוף ראשון יחיד, כאילו אתה אחד הסטודנטים.",
    # "כתוב את הסיכום בעברית בלבד, ללא תוספות לפני או אחרי הסיכום",
    # "בתשובתך אל תוסיף על הקלט ואל תגיב ואל תביע את דעתך. רק סכם בהתבסס על הקלט מהסטודנטים בלבד."
    # ])
    # system_instructions = dedent(f"""\
    #                 You are a student in an arabic language course. 
    #                 Your task is return concise summary of the opinion of the other students.
    #                 The students are answering the question: [<'{col_name.strip()}'>]
    #                 You are given list of students' opinions, separated by |, Summarize their opinions. 
    #                 Format your response with up to {min(max(math.ceil(math.log(len(sentences))),2), 5)} short sentences, phrased like a student's opinion.
    #                 Your response must be in English.
    #                 Do not answer the students, do not add your opinion, do not add comments before or after the summary. Just summarize!
    #                 """)
    num_output_sentences_ = min(max(math.ceil(math.log(len(sentences))),2), 5)
    system_instructions = dedent(f"""\
                    You are a student in an arabic language course. 
                    Your task is to summarize the opinion of the other students.
                    You are given list of students' opinions separated by |, Summarize their opinions. 
                    The students are answering the question: [<'{col_name.strip()}'>]
                    Format your response with up to {num_output_sentences_} short sentences, phrased like a student's opinion.
                    Your response must be in English. 
                    Focus on bringing the main common response that all students share and one rarer response not necessarily agreeing with the main common response.
                    Do not answer the students, do not add your opinion, do not add comments before or after the summary. Just summarize!
                    """)

        # "Ensure that the quote-examples are very different from each other and direct quotes from the input, in Hebrew."


    messages = [
            {
            'role': 'system',
            'content': system_instructions
            },
            {'role': 'user', 
                'content': sentences}
    ]
    if verbose>10:
        print(model)
        print(messages)
    response = ollama.chat(model, messages=messages)
    return response    


In [ ]:
def topic_info_to_keypoints(topic_row ,verbose=0):
    topic_row = str('.'.join([topic_row['Name'], str(set(topic_row['Representation'])), str(set(topic_row['Representative_Docs']))]))
    heb_eng_arab_numeric = re.compile(r'[^\u0590-\u05FF\u0600-\u06FFa-zA-Z]+')
    cleaned_topic_row = heb_eng_arab_numeric.sub(' ', str(topic_row))
    cleaned_topic_row = re.sub(r'\s+', '', cleaned_topic_row).strip()
    if verbose>9:
        print(cleaned_topic_row)

    messages = [
            {
            'role': 'system',
            'content':  'תפקידך למצוא עד חמש מילות מפתח של הטקסט ולהחזירן מופרדות בסימון נקודה. הקפד שכל מילה נבחרת תהיה מהטקסט הנתון ושהמילים תהיינה שונות אחת מן השניה. החזר לכל היותר חמש מילים שונות, בעברית בשורה אחת קצרה, ללא אף מילה נוספת לפני או אחרי, ללא מספור וללא מעבר שורה וללא הסבר נוסף.'
            },
            {'role': 'user', 
                'content': cleaned_topic_row}
    ]

    resp = ollama.chat(model='llama3.1:latest', messages=messages)
    if verbose>9:
        print(resp['message']['content'])
        print()
    return resp['message']['content']


In [ ]:
def plot_hierarchical_topics(topic_model, docs, save_to_file):
    hierarchical_topics = topic_model.hierarchical_topics(docs=docs)
    fig = topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)
    fig.write_html(save_to_file.with_suffix('.html'))
    print(f'Saved to {save_to_file}')


In [ ]:
def fit_transform_model_topic(col, verbose=0, hierarchical_plot_file=''):
    if verbose:
        print('='*80)
        print(f'{col.name} TOPIC MODELING')
        print('='*80)
    sentences = preprocess(col)
    topic_model = set_topic_model(verbose=verbose)
    topic_model.fit(sentences) # .fillna('NA')
    topics, prob = topic_model.transform(sentences)
    # representative_docs = topic_model.get_representative_docs()

    topic_info = topic_model.get_topic_info()
    topic_info['keywords'] = topic_info.apply(topic_info_to_keypoints, axis=1)
    topic_info.set_index('Topic', inplace=True)

    if hierarchical_plot_file:
        plot_hierarchical_topics(topic_model, docs=sentences, save_to_file=hierarchical_plot_file)

    sentences_with_topics = pd.DataFrame({'sentences':sentences, 
                                        'topic':topics})

    grouped_by_topics = sentences_with_topics.groupby('topic')['sentences'].agg([lambda x: list(set(x)),'size']).reset_index()
    grouped_by_topics.columns = ['topic','list', 'num_samples']
    grouped_by_topics['num_unique'] = grouped_by_topics['list'].apply(len)
    grouped_by_topics.set_index('topic', inplace=True)
    grouped_by_topics = grouped_by_topics.merge(topic_info, left_index=True, right_index=True)

    return grouped_by_topics, topic_model



In [ ]:
def summarize_topic(col_name, sentences, context_length=1000, verbose=True):
    s = '|'.join(sentences)
    topic_sub_responses = dict()
    if verbose:
        print('='*80)
        print(f'SUMMARIZATION. sentences example:')
        print(sentences[:3])
        print('='*80)
    cntr = 0
    for batch_start in range(0, len(s), context_length):
        # if cntr > 2:
        #     break
        cntr += 1
        batch_end = batch_start + 20 + context_length
        if verbose>3: print(f'{batch_start}-{batch_end} out of {len(s)}')
        sub_response = summarize(col_name, s[batch_start:batch_end], verbose=verbose)
        if verbose>3: print(sub_response['message']['content'], end='\n\n')
        topic_sub_responses[(batch_start,batch_end)] = sub_response
        # if batch_start > context_length:
        #     break

    clean_topic_subsummaries = [d_['message']['content'] for d_ in topic_sub_responses.values()]
    clean_topic_subsummaries = '|'.join(clean_topic_subsummaries).replace('\n','.').replace('•','.').replace('*','.')
    topic_summary = summarize(col_name, clean_topic_subsummaries, verbose=verbose)
    if verbose:
        print('='*80)
        print(f'SUMMARIZATION DONE:')
        print(f'{topic_summary['message']['content']=}')
        print('='*80)

    return topic_summary


In [ ]:

def topic_model_and_summarize_column(col_: pd.Series, save_to_folder: str|Path, verbose=0):
    col_name = re.sub(r'\W+', ' ', col_.name)
    col_name.strip()
    hierarchical_plot_file = save_to_folder/col_name
    sentences_col = preprocess(col_)
    grouped_by_topics, topic_model = fit_transform_model_topic(sentences_col, verbose=verbose, hierarchical_plot_file=hierarchical_plot_file)
    grouped_by_topics['summary'] = grouped_by_topics['list'].apply(lambda l : summarize_topic(col_.name, l, context_length=2500, verbose=verbose))
    grouped_by_topics['summary_content'] = grouped_by_topics['summary'].apply(lambda x: x['message']['content'].replace('\n',''))
    return grouped_by_topics


In [ ]:
for qstr_key, df in dfs.items():
    for col_name in text_columns[qstr_key]:
        print(col_name)


In [ ]:
import time
responses = dict()
cntr = 0
for qstr_key, df in dfs.items():
    if cntr > 2:
        break

    print(qstr_key)
    qstr_responses = dict()
    for col_name in text_columns[qstr_key]:
        cntr +=1
        if cntr > 2:
            break
        col = df[col_name].dropna()

        start = time.time()
        col_responses = topic_model_and_summarize_column(col, output_folder, verbose=7)
        qstr_responses[col_name] = col_responses
    responses[qstr_key] = qstr_responses


# Save/Load results to pickle or google spreadsheet

In [ ]:
import pickle

### Save to pickle
with open(output_folder/'responses.pkl', 'wb') as file:
    pickle.dump(responses, file)

###  Load from pickle
# with open(output_folder/'responses.pkl', 'rb') as file:
#     loaded_dfs = pickle.load(file)


In [ ]:
for qstr_key, v in responses.items():
    for question, summary in v.items():
        summary['summary_content'] = summary['summary'].apply(lambda x: x['message']['content'].replace('\n',''))

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
from gspread_dataframe import set_with_dataframe

def push_csv_to_google_spreadsheet2(credentials_json_path, df, spreadsheet_name, sheet_name):
    """
    Push data from multiple CSV files to different sheets in a Google Spreadsheet.

    :param credentials_json_path: Path to the JSON credentials file for the Google Service Account.
    :param spreadsheet_name: Name of the Google Spreadsheet.
    :param csv_file_paths: List of paths to CSV files.
    """
    # Define the scope
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]

    # Add your service account credentials
    creds = ServiceAccountCredentials.from_json_keyfile_name(credentials_json_path, scope)

    # Authorize the client
    client = gspread.authorize(creds)

    # Open the spreadsheet
    spreadsheet = client.open(spreadsheet_name)

    # Create a new sheet with the name derived from the CSV file
    clean_sheet_name = re.sub(r'\W+', ' ', sheet_name)
    short_sheet_name = clean_sheet_name[:30]
    try:
        sheet = spreadsheet.add_worksheet(title=short_sheet_name, rows=df.shape[0], cols=df.shape[1])
    except gspread.exceptions.APIError:
        # If the sheet already exists, get the existing sheet
        sheet = spreadsheet.worksheet(short_sheet_name)


    # Convert DataFrame to list of lists
    # data = [df.columns.values.tolist()] + df.values.tolist()

    # Clear the existing sheet content (if any)
    sheet.clear()

    sheet.append_row([sheet_name])
    set_with_dataframe(sheet, df, row=3)

    print("Data has been successfully pushed to the spreadsheet.")



In [ ]:

spreadsheet_name = experiement_name
credentials_json_path = '/Users/sl/dev/madrase/madrase-questionnaire-july-22-51ed22398a1d.json'

for qstr_key, v in responses.items():
    for question, summary in v.items():
        cols_to_csv = ['num_samples', 'num_unique', 'Count', 'Name', 'Representative_Docs', 'keywords' , 'summary_content']
        # summary[cols_to_csv].to_csv(output_folder/f'{question}.csv')
        worksheet_name = '.'.join([qstr_key,question])
        summary[['Representative_Docs', 'keywords']] = summary[['Representative_Docs', 'keywords']].astype(str)
        push_csv_to_google_spreadsheet2(credentials_json_path, 
                                        summary[cols_to_csv], 
                                        spreadsheet_name, 
                                        worksheet_name)
